In [117]:
import pandas as pd
from pandas.core.indexing import _IndexSlice

from pathlib import Path

idx: _IndexSlice = pd.IndexSlice

In [118]:
# Define the path to the data
DATA_PATH = Path("data/01_known_effects")

In [126]:
# Load the list of samples
samples = pd.read_csv(
    DATA_PATH / "experiments.csv",
    index_col=0
    )

# Create an index from the experiment table
samples_index_fields = {
        'Label': "label",
        # 'Effect in PSET',
        'Treatment': "treatment",
        'Experimenter, location': "experimenter_location",
        'Strain': "strain",
        'CO2 level': "CO2_level",
        'Cultivation + experiment temperature': "temperature",
        'Cultivation light intensity': "Light_intensity",
        'Dark or light acclimated': "light_acclimation",
        'Growth light color (nm)': "light_color",
        'Cultivator': "cultivator",
        'Medium': "medium",
        'Fluorometer': "fluorometer",
        'SP color (nm)': "SP_color",
        'SP intensity': "SP_intensity",
        'Measuring vessel': "vessel",
        'OD680 MC-1000': "OD680",
        'OD720 MC-1000': "OD720",
        'ΔOD': "deltaOD",
        'OD680/720 raw': "OD680_720",
}

samples_index = pd.MultiIndex.from_frame(
    samples.reset_index().loc[:, list(samples_index_fields.keys())]
)

In [127]:
# for i in samples_index_fields.values():
#     print(f"{i},")# : str | list[str | int] | slice | None = slice(None),")

# Create a function for easy indexing
def didx(
    label: str | list[str | int] | slice | None = slice(None),
    treatment: str | list[str | int] | slice | None = slice(None),
    experimenter_location: str | list[str | int] | slice | None = slice(None),
    strain: str | list[str | int] | slice | None = slice(None),
    CO2_level: str | list[str | int] | slice | None = slice(None),
    temperature: str | list[str | int] | slice | None = slice(None),
    Light_intensity: str | list[str | int] | slice | None = slice(None),
    light_acclimation: str | list[str | int] | slice | None = slice(None),
    light_color: str | list[str | int] | slice | None = slice(None),
    cultivator: str | list[str | int] | slice | None = slice(None),
    medium: str | list[str | int] | slice | None = slice(None),
    fluorometer: str | list[str | int] | slice | None = slice(None),
    SP_color: str | list[str | int] | slice | None = slice(None),
    SP_intensity: str | list[str | int] | slice | None = slice(None),
    vessel: str | list[str | int] | slice | None = slice(None),
    OD680: str | list[str | int] | slice | None = slice(None),
    OD720: str | list[str | int] | slice | None = slice(None),
    deltaOD: str | list[str | int] | slice | None = slice(None),
    OD680_720: str | list[str | int] | slice | None = slice(None),
) -> _IndexSlice:
    res: list[str | list[str | int] | slice | None] = [
        x
        for x in [
            label,
            treatment,
            experimenter_location,
            strain,
            CO2_level,
            temperature,
            Light_intensity,
            light_acclimation,
            light_color,
            cultivator,
            medium,
            fluorometer,
            SP_color,
            SP_intensity,
            vessel,
            OD680,
            OD720,
            deltaOD,
            OD680_720,
        ]
        if x is not None
    ]
    return idx[*res]

In [128]:
# Get the paths to the samples files
files = {}

for i in samples_index.get_level_values(0):
    _index = f"{i:04}"

    # Get the file to the current index
    file = list(DATA_PATH.glob(f"ojip_data/{_index}*"))[0]

    # Set the options for reading the data based on the used fluorometer
    if samples.loc[i, "Fluorometer"] == "MULTI-COLOR-PAM":
        skiprows = 0
        skipfooter = 0
        sep=";"
        index_col=0
        select_col = "Fluo, V"
    elif samples.loc[i, "Fluorometer"] == "AquaPen":
        skiprows = 7
        skipfooter = 38
        sep=r"\s"
        index_col=0
        select_col = "OJIP"
    else:
        print(i, samples.loc[i, "Fluorometer"])
        break

    # Read the data with the pre-defined options
    _df = pd.read_table(
        file,
        skiprows=skiprows,
        skipfooter=skipfooter,
        sep=sep,
        index_col=index_col,
        engine='c' if skipfooter == 0 else 'python'
    )[select_col]

    # Save the data
    files[i] = _df

# Concatenate the data
df = pd.DataFrame(files).sort_index(axis=1)
df.columns = samples_index

In [ ]:
df.loc[:, didx(fluorometer="MULTI-COLOR-PAM")].dropna().plot(legend=False)

In [ ]:

# # Create the DataFrame containing the data
# df = pd.DataFrame(
#     columns=samples_index,
#     col
# )